In [1]:
from openpyxl import load_workbook
import pandas as pd

In [2]:
df = pd.read_excel('kotlin-code-code.xlsx', sheet_name = 'data')

In [3]:
df.dtypes



category       object
resource-id    object
code           object
dtype: object

In [4]:
df['category'] = pd.Series(df['category'], dtype="string")
df['resource-id'] = pd.Series(df['resource-id'], dtype="string")
df['code'] = pd.Series(df['code'], dtype="string")
df = df.drop_duplicates()
df = df.sort_values(by=['resource-id'])

In [5]:
# final_data = []
# for index, row in df.iterrows():
#   final_data.append( row['category']+' '+row['resource-id'])


In [6]:
# final_data = pd.DataFrame(final_data)
# final_data.columns = ['context']
# final_data['context'] = pd.Series(final_data['context'], dtype="string")
# final_data = final_data.join(df['code'])

In [7]:
# final_data.drop_duplicates()

In [8]:
# final_data.dtypes
# len(final_data)
# final_data

In [9]:
df = df.reset_index(drop=True)

In [10]:
df

,category,resource-id,code
0,Science & Education,account_email,val accountEmail: TextView
1,Science & Education,account_email,override fun bindView(holder: AccountViewHolde...
2,Internet,addBookmarksPopupMenuItem,private fun createPopupMenu()
3,Internet,addBookmarksPopupMenuItem,private fun renderPopupMenus(browserShowing: B...
4,Internet,addFavoritePopupMenuItem,private fun createPopupMenu()
...,...,...,...
958,Time,week_day_name,private fun create4x2RemoteViews(context: Cont...
959,Time,week_day_name,private fun create2x2RemoteViews(context: Cont...
960,Time,week_day_name,private fun create4x1RemoteViews(context: Cont...
961,Internet,whitelistPopupMenuItem,private fun createPopupMenu()


In [11]:
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np

In [12]:
# X = df[['category', 'resource-id']]
X = df[[ 'resource-id']]
y = df['code']

In [13]:

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=42 )

X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

print(len(X_train), len(X_test), type(X_train))

770 193 <class 'pandas.core.frame.DataFrame'>


In [14]:
train_enc = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
train_enc.fit(X_train)
encoded_train_X = train_enc.transform(X_train)

test_enc = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
test_enc.fit( np.array(y_train).reshape(1, -1).transpose() )
encoded_train_y = test_enc.transform(  np.array(y_train).reshape(1, -1).transpose() )


In [15]:
len(y_train.unique())

609

In [16]:

from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from sklearn import tree

In [17]:
def get_model(max_depth, min_samples_leaf):

  #Configure the model
  model = tree.DecisionTreeClassifier(criterion = "gini", random_state = 43,
                                max_depth=max_depth, min_samples_leaf=min_samples_leaf)
  model = model.fit(encoded_train_X, encoded_train_y)

  return model

In [18]:
max_depths = [1, 3, 5, 10, 15]
min_samples_leaf = [5, 10, 15, 20, 25]
models_data = []

for i in max_depths:
  for j in min_samples_leaf:
    model = get_model(i, j)
    models_data.append( [i, j, model])

In [19]:
print(models_data)

[[1, 5, DecisionTreeClassifier(max_depth=1, min_samples_leaf=5, random_state=43)], [1, 10, DecisionTreeClassifier(max_depth=1, min_samples_leaf=10, random_state=43)], [1, 15, DecisionTreeClassifier(max_depth=1, min_samples_leaf=15, random_state=43)], [1, 20, DecisionTreeClassifier(max_depth=1, min_samples_leaf=20, random_state=43)], [1, 25, DecisionTreeClassifier(max_depth=1, min_samples_leaf=25, random_state=43)], [3, 5, DecisionTreeClassifier(max_depth=3, min_samples_leaf=5, random_state=43)], [3, 10, DecisionTreeClassifier(max_depth=3, min_samples_leaf=10, random_state=43)], [3, 15, DecisionTreeClassifier(max_depth=3, min_samples_leaf=15, random_state=43)], [3, 20, DecisionTreeClassifier(max_depth=3, min_samples_leaf=20, random_state=43)], [3, 25, DecisionTreeClassifier(max_depth=3, min_samples_leaf=25, random_state=43)], [5, 5, DecisionTreeClassifier(max_depth=5, min_samples_leaf=5, random_state=43)], [5, 10, DecisionTreeClassifier(max_depth=5, min_samples_leaf=10, random_state=43)

Replace extra characters

In [20]:
import re

def bleu_score( expected, actual ):

  expected_words_set = set( re.split('\s+', re.sub("[^0-9a-zA-Z]+", " ", expected)))
  actual_words_set = set( re.split('\s+', re.sub("[^0-9a-zA-Z]+", " ", actual)))

  total = max( len(expected_words_set), len(actual_words_set))
  return len( expected_words_set.intersection(actual_words_set) ) / total

In [21]:
encoded_test_X = train_enc.transform(X_test)
encoded_test_y = test_enc.transform( np.array(y_test).reshape(1, -1).transpose() )
# print( encoded_test_X.shape, encoded_test_y.shape)

In [22]:


def get_result(model):
  temp = []
  counter = 0

  blue_scores = []

  for index, data in enumerate(encoded_test_X):
    # print( data )
    
    result = model.predict_proba( pd.DataFrame(data).transpose() ).transpose()
    k = 5
    top_k_index = np.argsort(result, axis=0, kind='heap', order=None)[-k:][::-1]

    max_score = 0.0

    # print(np.array(top_k_index), y_test[index])
    # print(test_enc.inverse_transform( np.array(top_k_index) ).flatten())
    for idx in top_k_index:
      idx = encoded_train_y[idx]
      expected:str = test_enc.inverse_transform( np.array(idx).reshape(-1, 1) ).flatten()[0]
      actual:str = y_test[index]

      max_score = max( max_score,  bleu_score(expected, actual) ) 
      if test_enc.inverse_transform( np.array(idx).reshape(-1, 1) ).flatten()[0] == y_test[index]:
        counter = counter + 1

    blue_scores.append(max_score)

  accuracy = counter / len(X_test)
  score = sum(blue_scores) / len(blue_scores)

  return [accuracy, score]

In [23]:
print(len(models_data))
for model_data in models_data:
  model_data.extend( get_result(model_data[2]))

25


In [24]:
for row in models_data:
  row.pop(2)
  print(row)

[1, 5, 0.0, 0.3926597760542001]
[1, 10, 0.0, 0.3875401116081109]
[1, 15, 0.0, 0.3875401116081109]
[1, 20, 0.0, 0.3875401116081109]
[1, 25, 0.0, 0.3875401116081109]
[3, 5, 0.010362694300518135, 0.3928037023639296]
[3, 10, 0.0, 0.3948186707001415]
[3, 15, 0.0, 0.39222799712501194]
[3, 20, 0.0, 0.39222799712501194]
[3, 25, 0.0, 0.3916522918860943]
[5, 5, 0.0, 0.3698659607733383]
[5, 10, 0.0051813471502590676, 0.37336337009976317]
[5, 15, 0.0, 0.3819916589093992]
[5, 20, 0.0, 0.38020697266875436]
[5, 25, 0.0, 0.381790162075778]
[10, 5, 0.010362694300518135, 0.3636100261495281]
[10, 10, 0.015544041450777202, 0.37789325299752186]
[10, 15, 0.0051813471502590676, 0.3792923863914738]
[10, 20, 0.0051813471502590676, 0.37616918547034545]
[10, 25, 0.0, 0.37568376128025793]
[15, 5, 0.010362694300518135, 0.3624697731958037]
[15, 10, 0.015544041450777202, 0.37841138771254784]
[15, 15, 0.0051813471502590676, 0.3792923863914738]
[15, 20, 0.0051813471502590676, 0.37616918547034545]
[15, 25, 0.0, 0.37568

Decode Values

In [25]:
result_df = pd.DataFrame(models_data)

In [26]:
result_df.columns = ['Max Depths', 'Min Samples/Leaf', 'Accuracy', 'Bleu Score']

In [27]:
result_df.to_excel(excel_writer='result_decision_tree_without_category.xlsx', sheet_name='data')

In [ ]:
!pip install hvplot

In [29]:
import numpy as np
# import hvplot.pandas
import pandas as pd

result_df.plot(x='Max Depths', y=['Accuracy', 'Bleu Score',  ], kind='scatter', backend='plotly', 
    width=500,   height = 400
)



In [30]:
result_df.plot(x='Min Samples/Leaf', y=['Accuracy', 'Bleu Score',  ], kind='scatter', backend='plotly', 
    width=500,   height = 400
)

In [31]:
result_df

,Max Depths,Min Samples/Leaf,Accuracy,Bleu Score
0,1,5,0.000000,0.392660
1,1,10,0.000000,0.387540
2,1,15,0.000000,0.387540
3,1,20,0.000000,0.387540
4,1,25,0.000000,0.387540
5,3,5,0.010363,0.392804
6,3,10,0.000000,0.394819
7,3,15,0.000000,0.392228
8,3,20,0.000000,0.392228
9,3,25,0.000000,0.391652
